# 首先对于大图，需要将其切成小图，放在tmp中

In [2]:
import sys
import cv2
import shutil
import os
sys.path.append('/jinkevi')
from common.el_process import  grid_cut 
from common.el_precut import ElPreCuter

cuter = ElPreCuter
testimg  = cv2.imread('../PA03181010804974.jpg')
testimg  = cuter().get_precut(testimg,False)
dataset  = grid_cut(testimg,6,12,True)

if os.path.exists('../data/testimg/'):
    shutil.rmtree('../data/testimg/')
os.mkdir('../data/testimg/')
for iidx  , i in enumerate(dataset[1][0]):
    for  jidx ,j in enumerate( dataset[1][1] ):
        if i != dataset[1][0][-1] and j != dataset[1][1][-1]:
            img = dataset[0][(i,j)]
            cv2.imwrite(
                os.path.join('../data/testimg/' + str(iidx) + '_' + str(jidx) + '.jpg'),img
            )

# 从文件夹中取出小图进行测试 

In [ ]:
#!/usr/bin/env python2

# Copyright (c) 2017-present, Facebook, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
##############################################################################

"""Perform inference on a single image or all images with a certain extension
(e.g., .jpg) in a folder.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from collections import defaultdict
import argparse
import cv2  # NOQA (Must import before importing caffe2 due to bug in cv2)
import glob
import logging
import os
import sys
import time
import pickle 

from caffe2.python import workspace

from core.config import assert_and_infer_cfg
from core.config import cfg
from core.config import merge_cfg_from_file
from utils.timer import Timer
import core.test_engine as infer_engine
import datasets.dummy_datasets as dummy_datasets
import utils.c2 as c2_utils
import utils.logging
import utils.vis as vis_utils

import numpy as np
import sys
sys.path.append('/jinkevi/el_server/')
import  post_process.yinlie_utils.extract_yinlie_feature as fy

from easydict import EasyDict as easy


c2_utils.import_detectron_ops()
# OpenCL may be enabled by default in OpenCV3; disable it because it's not
# thread safe and causes unwanted GPU memory allocations.
cv2.ocl.setUseOpenCL(False)

import  matplotlib.pyplot  as plt
%matplotlib inline

import time


def testimg(args):
    logger = logging.getLogger(__name__)
    merge_cfg_from_file(args.cfg)
    cfg.TEST.WEIGHTS = args.weights
    cfg.NUM_GPUS = 1
    assert_and_infer_cfg()
    model = infer_engine.initialize_model_from_cfg()
    dummy_coco_dataset = dummy_datasets.get_coco_dataset('yinlie')

    if os.path.isdir(args.im_or_folder):
        im_list = glob.iglob(args.im_or_folder + '/*.' + args.image_ext)
    else:
        im_list = [args.im_or_folder]
    print('we now start to detect '  , args.im_or_folder + '/*.' + args.image_ext)

    
    count  = 0 
    for i, im_name in enumerate(im_list):

        out_name = os.path.join(
            args.output_dir, '{}'.format(os.path.basename(im_name) + '.pdf')
        )

        im = cv2.imread(im_name)
        timers = defaultdict(Timer)
        t = time.time()
        with c2_utils.NamedCudaScope(0):
            
            start  = time.time()
            cls_boxes, cls_segms, cls_keyps = infer_engine.im_detect_all(
                model, im, None, timers=timers
            )
            end = time.time()
            
            a = cls_boxes[1][:,4][0]   if cls_boxes[1].shape[0] != 0 else 0
            b = cls_boxes[2][:,4][0]   if cls_boxes[2].shape[0] != 0 else 0
            
            if a > 0.985:
                count +=1 

                xmin , ymin, xmax ,ymax = cls_boxes[1][0][:4].astype(int)
                  
                
                jiaYinlieBx = [xmin , ymin, xmax ,ymax ]
                subimg = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
                shanxians = fy.find_shanxian_row(subimg)[1]

                
                yinlieMask, shanxian, labels3 = fy.getYinlieInfo(subimg,jiaYinlieBx,verbose = False,padding = 10)
                status , l1 , mask   = fy.getThicknessInfo(subimg, jiaYinlieBx ,shanxian, labels3, verbose= False , thick=3,thres = 10)
                

                # 然后获得焊点的外围
                inner_xmin = 15
                inner_xmax = subimg.shape[1] - 15
                inner_ymin = shanxians[0] // 3
                inner_ymax = shanxians[-1] + (subimg.shape[0] - shanxians[-1] ) *2 // 3

                subimg  = cv2.rectangle(subimg, (xmin,ymin), (xmax,ymax), (0,255,0), 2)
                subimg  = cv2.rectangle(subimg, (inner_xmin , inner_ymin) ,(inner_xmax,inner_ymax) ,(0,255,0) ,2)
                
                if status == False:
                    count +=1 
                    
                    # 执行第二阶段
                    print('-'*80)
                    print('count:' , end - start)
                    print(count)
                    print(im_name)
                    print('l1 is ', l1)
                    print(cls_boxes[1][:,4][0])

                    print('compare with shanxian : ',(ymax - ymin ) > np.diff(shanxians ).mean())
                    print('diff of y is ' ,ymax - ymin)
                    print('shanxian diff is ' ,0.7 * np.diff(shanxians ).mean())
                    print('if large yilie',(ymax - ymin ) > 0.8 * np.diff(shanxians ).mean())


                    plt.figure(figsize = (10,10))
                    plt.title(status)
                    plt.imshow(subimg,'gray')
                    plt.show()



if __name__ == '__main__':
    workspace.GlobalInit(['caffe2', '--caffe2_log_level=0'])
    utils.logging.setup_logging(__name__)
    args = easy()
    args.cfg = '/jinkevi/el_server/detectron/configs/EL_configs/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN_yinlie.yaml'
    args.weights = '/tmp/output/model_yinlie.pkl'
    args.output_dir = '/tmp/output/testimg_output'
    args.im_or_folder = '../data/testimg/' 
    args.image_ext = 'jpg'
    testimg(args)

Found Detectron ops lib: /usr/local/caffe2_build/lib/libcaffe2_detectron_ops_gpu.so
WARNING cnn.py:  40: [====DEPRECATE WARNING====]: you are creating an object from CNNModelHelper class which will be deprecated soon. Please use ModelHelper object with brew module. For more information, please refer to caffe2.ai and python/brew.py, python/brew_test.py for more information.
INFO net.py:  57: Loading weights from: /tmp/output/model_yinlie.pkl
we now start to detect  ../data/testimg//*.jpg
--------------------------------------------------------------------------------
count: 0.0355401039124
2
../data/testimg/2_0.jpg
l1 is  2.1190897751506874
0.9883605
compare with shanxian :  True
diff of y is  161
shanxian diff is  55.125
if large yilie True


78.69006752597979